In [ ]:
import torch 
import pandas as pd

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Running on {device}.")

In [ ]:
train_url = "https://raw.githubusercontent.com/PhilChodrow/ml-notes/main/data/sign-language-mnist/sign_mnist_train.csv"
test_url = "https://raw.githubusercontent.com/PhilChodrow/ml-notes/main/data/sign-language-mnist/sign_mnist_test.csv"

df_train = pd.read_csv(train_url)
df_val   = pd.read_csv(test_url)

def prep_data(df):
    n, p = df.shape[0], df.shape[1] - 1
    y = torch.tensor(df["label"].values)
    X = df.drop(["label"], axis = 1)
    X = torch.tensor(X.values)
    X = torch.reshape(X, (n, 1, 28, 28))
    X = X / 255

    # important: move the data to GPU if available
    X, y = X.to(device), y.to(device)

    return X, y

X_train, y_train = prep_data(df_train)
X_val, y_val     = prep_data(df_val)

In [ ]:
data_loader_train = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(X_train, y_train),
    batch_size = 512,
    shuffle = True
)

data_loader_val = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(X_val, y_val),
    batch_size = 512,
    shuffle = True
)

In [ ]:
import torch.nn as nn

class Encoder(nn.Module):

    def __init__(self):
        super().__init__()

        self.pipeline = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 4, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

    def forward(self, x):
        return self.pipeline(x)

class Decoder(nn.Module):
    
    def __init__(self):
        super().__init__()

        self.pipeline = nn.Sequential(
            nn.ConvTranspose2d(4, 16, 
                               kernel_size=3, 
                               stride=2, 
                               padding=1, 
                               output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 1, 
                               kernel_size=3, 
                               stride=2, 
                               padding=1, 
                               output_padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.pipeline(x)


encoder = Encoder().to(device)
decoder = Decoder().to(device)

X, y = next(iter(data_loader_train))

s_hat = encoder(X)
X_hat = decoder(s_hat)

print(X.size())
print(s_hat.size())
print(X_hat.size())

In [ ]:
loss_fn = nn.MSELoss()

opt = torch.optim.Adam(
    list(encoder.parameters()) + list(decoder.parameters()),
    lr = 0.1
)

for epoch in range(10):
    for X, y in data_loader_train:
        opt.zero_grad()

        s_hat = encoder(X)
        X_hat = decoder(s_hat)

        loss = loss_fn(X, X_hat)

        loss.backward()
    print(f"Final batch in epoch {epoch}: {loss.item()}")

In [ ]:
s_hat.size()

In [ ]:
from matplotlib import pyplot as plt
X, y = next(iter(data_loader_train))

i = 0
x = X[i,:,:,:]

x_hat = decoder(encoder(x))

fig, ax = plt.subplots(1,2, figsize = (6, 3))

ax[0].imshow(x_hat[0,:,:].detach(), cmap = "Greys")
ax[1].imshow(x[0,:,:].detach(), cmap = "Greys")